# **ALARAJOY Library Conversion QA Notebook**


This Jupyter Notebook is designed to enable comparisons for neutron activation responses of a given single parent element as calculated by ALARA for the purpose of validating ALARAJOY-processed data from the FENDL3.2x data sets.

**Import Packages**

In [10]:
import alarajoy_QA as qa
import importlib

**Run ALARA with each prepared binary data library**

In [ ]:
importlib.reload(qa)
data_soure = {
    'fendl2' : '/groupspace/shared/n/nukecode/ALARA/data/fendl2bin',
    'fendl3' : '../../examples/data/fendl3'
}

element = input('Select single parent element to evaluate: ').lower()

for libname, binary in data_soure.items():
    alara_input = qa.fill_alara_template(element, binary)
    qa.write_alara_input_file(alara_input)
    output = qa.run_alara(element,libname)
    data_soure[libname] = output